In [10]:
import serial
import json
import time
import pandas as pd

PORT = "COM5"  
BAUD = 115200

def read_logs():
    logs = []
    with serial.Serial(PORT, BAUD, timeout=5) as ser:
        time.sleep(2) 
        ser.write(b"GET_LOGS\n")
        buffer = ""
        start = time.time()
        while time.time() - start < 10:
            if ser.in_waiting:
                line = ser.readline().decode('utf-8', errors='ignore').strip()
                if line:
                    buffer += line
        try:
            for part in buffer.split('}{'):
                if not part.startswith('{'):
                    part = '{' + part
                if not part.endswith('}'):
                    part = part + '}'
                logs.append(json.loads(part))
        except json.JSONDecodeError as e:
            print("JSON decode error:", e)
    return logs

def logs_to_dataframe(logs):
    records = []
    for log in logs:
        timestamp = log.get("T")
        entries = log.get("entries", [])
        if isinstance(entries, list):
            for entry in entries:
                records.append({
                    "timestamp": timestamp,
                    "anchor": entry.get("A"),
                    "range": entry.get("R"),
                    "rx_power": entry.get("Rx")
                })
    return pd.DataFrame(records)

In [11]:
logs = read_logs()
df = logs_to_dataframe(logs)
print(df)

SerialException: [Errno 2] could not open port COM5: [Errno 2] No such file or directory: 'COM5'